In [3]:
from openai import OpenAI
import os
from enum import Enum
import json
from pydantic import BaseModel, Field  # For structured data validation
from typing import List, Literal, Optional


client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key="voc-1142391097160736424292768839a5d4b7a92.77641675")

In [4]:
# Define sample FNOL texts
# TODO: [Optional] Add more sample FNOL texts to test various scenarios

sample_fnols = [
    """
    Claim ID: C001
    Customer: John Smith
    Vehicle: 2018 Toyota Camry
    Incident: While driving on the highway, a rock hit my windshield and caused a small chip
    about the size of a quarter. No other damage was observed.
    """,
    """
    Claim ID: C002
    Customer: Sarah Johnson
    Vehicle: 2020 Honda Civic
    Incident: I was parked at the grocery store and returned to find someone had hit my car and
    dented the rear bumper and taillight. The taillight is broken and the bumper has a large dent.
    """,
    """
    Claim ID: C003
    Customer: Michael Rodriguez
    Vehicle: 2022 Ford F-150
    Incident: I was involved in a serious collision at an intersection. The front of my truck is
    severely damaged, including the hood, bumper, radiator, and engine compartment. The airbags
    deployed and the vehicle is not drivable.
    """,
    """
    Claim ID: C004
    Customer: Emma Williams
    Vehicle: 2019 Subaru Outback
    Incident: My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and
    trunk. The side mirrors were also damaged and one window has a small crack.
    """,
    """
    Claim ID: C005
    Customer: David Brown
    Vehicle: 2021 Tesla Model 3
    Incident: Someone keyed my car in the parking lot. There are deep scratches along both doors
    on the driver's side.
    """,
]

In [18]:
# validation template
class ClaimInformation(BaseModel):
    claim_id: str = Field(..., min_length=2, max_length=10)
    name: str = Field(..., min_length=2, max_length=100)
    vehicle: str = Field(..., min_length=2, max_length=100)
    loss_desc: str = Field(..., min_length=10, max_length=500)
    damage_area: List[
        Literal[
            "windshield",
            "front",
            "rear",
            "side",
            "roof",
            "hood",
            "door",
            "bumper",
            "fender",
            "quarter panel",
            "trunk",
            "glass",
        ]
    ] = Field(..., min_items=1)

def validate_etxracted_info(claim_info_json: str) -> ClaimInformation:
    try:
        # Parse the JSON string
        claim_info_dict = json.loads(claim_info_json)
        # Validate with Pydantic model
        validated_info = ClaimInformation(**claim_info_dict)
        return validated_info
    except Exception as e:
        raise ValueError(f"Gate 1 validation failed: {str(e)}")

In [14]:
info_extraction_system_prompt = """
    You are an auto insurance claim processing assistant. Your task is to extract key information from First Notice of Loss (FNOL) reports.
    
    Format your response as a valid JSON object with the following keys:
    - claim_id (str): The claim ID
    - name (str): The customer's full name
    - vehicle (str): The vehicle make, model, and year
    - loss_desc (str): A concise description of the incident
    - damage_area (list[str]): A list of damaged areas on the vehicle (at least one of:
        - windshield
        - front
        - rear
        - side
        - roof
        - hood
        - door
        - bumper
        - fender
        - quarter panel
        - trunk
        - glass
    
    For damage_area, only use items from the list above.
    
    Only respond with the JSON object, nothing else.
"""

In [15]:

def extract_claim_info(fnol_text):
    messages = [
        {"role": "system", "content": info_extraction_system_prompt},
        {"role": "user", "content": fnol_text},
    ]

    response = get_completion(messages=messages)

    try:
        validated_info = validate_etxracted_info(response)
        return validated_info
    except ValueError as e:
        print(f"Gate 1 failed: {e}")
        return None

In [19]:
extract_claim_info(sample_fnols[1])

ClaimInformation(claim_id='C002', name='Sarah Johnson', vehicle='2020 Honda Civic', loss_desc='Someone hit my car while it was parked, damaging the rear bumper and taillight.', damage_area=['rear', 'bumper', 'glass'])

In [22]:

class SeverityAssessment(BaseModel):
    severity: Literal["Minor", "Moderate", "Major"]
    est_cost: float = Field(..., gt=0)


severity_assessment_system_prompt = """
    You are an auto insurance damage assessor. Your task is to evaluate the severity of vehicle damage and estimate repair costs.
    
    Apply these carrier heuristics:
    - Minor damage: Small dents, scratches, glass chips (cost range: $100-$1,000)
    - Moderate damage: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
    - Major damage: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)
    
    Based on the claim information provided, determine:
    1. Severity level (Minor, Moderate, or Major)
    2. Estimated repair cost (in USD)
    
    Format your response as a valid JSON object with the following keys:
    - severity: One of "Minor", "Moderate", or "Major"
    - est_cost: Numeric estimate of repair costs (e.g., 750.00)
    
    Only respond with the JSON object, nothing else.
"""

In [25]:
def cost_range_ok(severity_json: str) -> SeverityAssessment:
    try:
        # Parse the JSON string
        severity_dict = json.loads(severity_json)
        validated_severity = SeverityAssessment(**severity_dict)

        # Check cost range based on severity
        if validated_severity.severity == "Minor" and (
            validated_severity.est_cost < 100 or validated_severity.est_cost > 1000
        ):
            raise ValueError(
                f"Minor damage should cost between $100-$1000, got ${validated_severity.est_cost}"
            )
        elif validated_severity.severity == "Moderate" and (
            validated_severity.est_cost < 1000 or validated_severity.est_cost > 5000
        ):
            raise ValueError(
                f"Moderate damage should cost between $1000-$5000, got ${validated_severity.est_cost}"
            )
        elif validated_severity.severity == "Major" and (
            validated_severity.est_cost < 5000 or validated_severity.est_cost > 50000
        ):
            raise ValueError(
                f"Major damage should cost between $5000-$50000, got ${validated_severity.est_cost}"
            )

        return validated_severity
    except Exception as e:
        raise ValueError(f"Gate 2 validation failed: {str(e)}")


def assess_severity(claim_info: ClaimInformation) -> Optional[SeverityAssessment]:
    # Convert Pydantic model to JSON string
    claim_info_json = claim_info.model_dump_json()

    messages = [
        {"role": "system", "content": severity_assessment_system_prompt},
        {"role": "user", "content": claim_info_json},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the severity assessment
    try:
        validated_severity = cost_range_ok(response)
        return validated_severity
    except ValueError as e:
        print(f"Gate 2 failed: {e}. Response: {response}")
        return None

In [26]:
claim = extract_claim_info(sample_fnols[1])
severity_assessment = assess_severity(claim)

In [27]:
print(claim)
print(severity_assessment)

claim_id='C002' name='Sarah Johnson' vehicle='2020 Honda Civic' loss_desc='Hit while parked at the grocery store, causing damage to the rear bumper and taillight.' damage_area=['rear', 'bumper', 'glass']
severity='Moderate' est_cost=2500.0
